# Run Test Method

In [8]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Modelo

In [9]:
from __future__ import annotations
import ast, sys
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Optional

@dataclass(frozen=True)
class CodeWarning:
    message: str
    lineno: int
    col: int
    end_lineno: Optional[int] = None
    end_col: Optional[int] = None

class DetectionRule:
    name: str = "Generic Detector"
    def search(self, src: str) -> List[CodeWarning]:
        raise NotImplementedError

class Patcher:
    name: str = "Generic Patcher"
    def patch(self, tree: ast.AST) -> ast.AST:
        raise NotImplementedError

class DCCLinter:
    def __init__(self, detectors: Iterable[DetectionRule], patchers: Iterable[Patcher]):
        self.detectors = detectors
        self.patchers = patchers

    def search(self, src: str) -> List[CodeWarning]:
        out: List[CodeWarning] = []
        for d in self.detectors:
            out.extend(d.search(src))
        return sorted(out, key=lambda w: (w.lineno, w.col))

    def patch(self, src: str) -> str:
        tree = ast.parse(src)
        for p in self.patchers:
          tree = p.patch(tree)
        return ast.unparse(tree)

# None Compares

## Detector

In [10]:
class MapFilterDetectionRule(DetectionRule):
    name = "Detect list(map(...)) / list(filter(...))"

    def search(self, src: str) -> List[CodeWarning]:
        # Detecta list(map|filter(...)).
        tree = ast.parse(src)
        warnings: List[CodeWarning] = []
        for node in ast.walk(tree):
            if isinstance(node, ast.Call) and isinstance(node.func, ast.Name) and node.func.id == 'list' and node.args:
                inner = node.args[0]
                if isinstance(inner, ast.Call) and isinstance(inner.func, ast.Name) and inner.func.id in ('map','filter'):
                    warnings.append(CodeWarning("Usa comprensión de listas en vez de list(...)", node.lineno, node.col_offset))
                    warnings.append(CodeWarning(f"Evita el uso de {inner.func.id} dentro de list", inner.lineno, inner.col_offset))
        return warnings

## Patcher

In [11]:
class MapFilterPatcher(Patcher):
    name: str = "Map/Filter to List Comprehension"

    def patch(self, tree: ast.AST) -> ast.AST:
        # Convierte list(map|filter(...)) a comprensión de listas.
        class T(ast.NodeTransformer):
            def visit_Call(self, node):
                if (isinstance(node.func, ast.Name) and node.func.id == 'list'
                    and node.args and isinstance(node.args[0], ast.Call)
                    and isinstance(node.args[0].func, ast.Name)
                    and node.args[0].func.id in ('map','filter')):
                    inner = node.args[0]
                    if inner.func.id == 'map' and len(inner.args) == 2:
                        f, it = inner.args
                        return ast.ListComp(
                            elt=ast.Call(func=f, args=[ast.Name('_x', ast.Load())], keywords=[]),
                            generators=[ast.comprehension(target=ast.Name('_x', ast.Store()), iter=inner.args[1], ifs=[], is_async=0)]
                        )
                    if inner.func.id == 'filter' and len(inner.args) == 2:
                        pred, it = inner.args
                        var = ast.Name('_x', ast.Load())
                        return ast.ListComp(
                            elt=var,
                            generators=[ast.comprehension(target=ast.Name('_x', ast.Store()), iter=it, ifs=[ast.Call(func=pred, args=[var], keywords=[])], is_async=0)]
                        )
                return self.generic_visit(node)
        return ast.fix_missing_locations(T().visit(tree))

## Tests

In [12]:
import unittest

class TestMapFilter(unittest.TestCase):
    def setUp(self):
        self.linter = DCCLinter([MapFilterDetectionRule()],[MapFilterPatcher()])

    def test_map(self):
        s = "xs = list(map(str, [1,2]))\n"
        warnings = self.linter.search(s)
        self.assertEqual(len(warnings), 2)

    def test_filter(self):
        s = "ys = list(filter(lambda x: x>0, [-1,0,1]))\n"
        out = self.linter.patch(s)
        self.assertIn("[_x for _x in [-1, 0, 1] if (lambda x: x > 0)(_x)]", out)

    # Tests extra
    def test_map_patch(self):
        s = "xs = list(map(str, [1,2]))\n"
        out = self.linter.patch(s)
        self.assertIn("[str(_x) for _x in [1, 2]]", out)

    def test_map_lambda_patch(self):
        s = "ys = list(map(lambda n: n+1, [1,2,3]))\n"
        out = self.linter.patch(s)
        self.assertIn("[(lambda n: n + 1)(_x) for _x in [1, 2, 3]]", out)

    def test_no_warning_for_list_comp(self):
        s = "xs = [x for x in [1,2,3]]\n"
        self.assertEqual(len(self.linter.search(s)), 0)

In [13]:
runTests(TestMapFilter)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK
....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


## Real World Code

In [14]:
def find_cell(comment):
    """
    Busca la primera celda en el cuaderno actual cuya primera línea
    comience con el comentario dado (por ejemplo, "# Case Study 1").

    Parámetros:
        comment (str): El comentario que se quiere buscar.

    Retorna:
        str | None: El código fuente de la celda encontrada, o None si no existe.
    """
    for cell_source in In:  # `In` contiene el código fuente de todas las celdas ejecutadas
        lines = cell_source.strip().splitlines()
        if lines and lines[0].strip().startswith(comment):  # revisa la primera línea
            return cell_source
    return None

In [15]:
# Case Study 1
def process_names(names):
    # Convierte todos los nombres a mayúsculas
    upper_names = list(map(str.upper, names))

    # Filtra los nombres que empiezan con "A"
    a_names = list(filter(lambda n: n.startswith("A"), upper_names))

    return a_names

def normalize_numbers(nums):
    # Convierte a enteros
    int_nums = list(map(int, nums))

    # Filtra solo los números positivos
    positives = list(filter(lambda n: n > 0, int_nums))

    # Aplica un cálculo adicional
    doubled = list(map(lambda n: n * 2, positives))

    return doubled

# Dentro de un if
nums = ["1","-2","3"]
if len(nums) > 0:
    ints_nonneg = list(filter(lambda x: int(x) >= 0, nums))

# En un bucle
results = []
for batch in [[1,2,3],[4,5,6]]:
    results.append(list(map(lambda n: n*n, batch)))

# En una función con predicado nombrado
def non_empty(strings):
    return list(filter(lambda s: len(s) > 0, strings))


In [16]:
linter = DCCLinter([MapFilterDetectionRule()],[MapFilterPatcher()])
bad_code =  find_cell("# Case Study 1")
warnings = linter.search(bad_code)
print("-= Warnings =-")
for w in warnings:
    print(w)
good_code = linter.patch(bad_code)
print("-= Fixed Code =-")
print(good_code)

-= Warnings =-
CodeWarning(message='Usa comprensión de listas en vez de list(...)', lineno=4, col=18, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de map dentro de list', lineno=4, col=23, end_lineno=None, end_col=None)
CodeWarning(message='Usa comprensión de listas en vez de list(...)', lineno=7, col=14, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de filter dentro de list', lineno=7, col=19, end_lineno=None, end_col=None)
CodeWarning(message='Usa comprensión de listas en vez de list(...)', lineno=13, col=15, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de map dentro de list', lineno=13, col=20, end_lineno=None, end_col=None)
CodeWarning(message='Usa comprensión de listas en vez de list(...)', lineno=16, col=16, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de filter dentro de list', lineno=16, col=21, end_lineno=None, end_col=None)
CodeWarning(message='Usa comprensión de listas en vez de list(...)', lineno